### Class-based Decorators

It's also possible to implement a decorator using classes, in fact, any decorator that you can implement as a function can be done with a class instead. <br><br>

But then, why would you do this? Basically, for certain kinds of more complex decorators, classes are better suited, more readable or easier to work with. <br><br>

The secret to decorating with classes is the magic method:

    __call__

Any object can implement \_\_call\_\_ to make it callable, meaning the object can be called like a function. A simple example follows:



In [2]:
class Prefixer:
    def __init__(self,prefix):
        self.prefix = prefix
    def __call__(self,message):
        return self.prefix + message

migosays = Prefixer("Simion says: ")
migosays("Get up and dance!")

'Simion says: Get up and dance!'

Just looking at simonsays("Get up and dance!") in isolation, you’d never guess it is
anything other than a normal function. In fact, it’s an instance of Prefixer.<br><br>

You can use \_\_call\_\_ to implement decorators, in a very different way.<br><br>

Quiz yourself: 

    using this information about __call__, how might you implement printlog as a class instead of a function?

In [3]:
class Printlog:
    def __init__(self,func):
        self.func = func
    
    def __call__(self,*args,**kwargs):
        print("Calling: {}".format(self.func.__name__))
        return self.func(*args,**kwargs)


# compare to the function version
def printlog(func):
    def wrapper(*args,**kwargs):
        print("Calling: {}".format(func.__name__))
        return func(*args,**kwargs)
    return wrapper

In [5]:
@Printlog
def foo(x):
    print(x+2)
    
@Printlog
def baz(x,y):
    return x+y

foo(7)


Calling: foo
9


In [6]:
baz(3,2)

Calling: baz


5

The above is the same as below

In [7]:
def bar(x):
    return x**2

Printlog(bar)(5)


Calling: bar


25

Class-based decorators have a few advantages over function-based. 

For one thing, the decorator is a class, which means 

you can leverage inheritance. For example:

In [9]:
import sys

class ResultAnnouncer:
    stream  = sys.stdout
    prefix = "RESULT"
    def __init__(self,func):
        self.func = func
        
    def __call__(self,*args,**kwargs):
        value = self.func(*args,**kwargs)
        self.stream.write("{}: {}\n".format(self.prefix,value))
        return value

class StdErrResultAnnouncer(ResultAnnouncer):
    stream = sys.stderr
    prefix= "ERROR"

Another benefic is when you prefer to accumulate state in object attribute.<br>

For example, the Countcalls function decorator above could be implemented as a class



In [10]:
class CountCalls:
    def __init__(self,func):
        self.func = func
        self.count = 1
        
    def __call__(self,*args,**kwargs):
        print("# of calls: {}".format(self.count))
        self.count +=1 
        return self.func(*args,**kwargs)
    
@CountCalls
def foo(x):
    return x+2


In [11]:
foo(5)

# of calls: 1


7

In [12]:
foo(3)

# of calls: 2


5

Notice this also lets us access foo.count, if we want to check the count outside the decorated function, it's such a nice way. 

In [19]:
foo(5)

# of calls: 3


7

In [20]:
foo.count

4

### How to pass argument just as in function decoration?

In this case, the constructor accepts not the func object to be decorated but the parameter on the decorator line. <br><br>

The \_\_call\_\_ method must take the ***func*** object, define a wrapper function and return it. 

In [21]:
class Add:
    def __init__(self,increment):
        self.increment = increment
    
    def __call__(self,func):
        def wrapper(*args,**kwargs):
            return func(*args,**kwargs) + self.increment
        return wrapper


In [22]:
@Add(2)
def foo(x):
    return x**2

foo(5)

27

In [23]:
@Add(4)
def bar(n):
    return n*2

bar(77)

158

As you can see, any function-based decorator can be implemented as a class-based one.
<br><br>

But it's possible to design class-based decorators which CANNOT be translated into a function-based form.<br><br>


